In [1]:
import panel as pn
import param
from panel.reactive import ReactiveHTML

In [2]:
class LeafletScatterFull(ReactiveHTML):
    """Leaflet map with scatter, hover marker, layer control, measuring + drawing tools."""

    # Data/state
    center = param.XYCoordinates(default=(37.0, -122.0))
    zoom   = param.Integer(default=11, bounds=(1, 22))
    lats   = param.List(default=[])
    lons   = param.List(default=[])
    values = param.List(default=[])
    cmap   = param.String(default="Viridis256")
    marker = param.XYCoordinates(default=None)
    show_marker = param.Boolean(default=False)

    # Container and map options
    container_style = param.String(default="width:100%;height:100%;")
    map_options     = param.Dict(default={})
    tile_layers     = param.Dict(default={
        "OSM": ("https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png",
                {"attribution": "&copy; OSM contributors"}),
        "Topo": ("https://{s}.tile.opentopomap.org/{z}/{x}/{y}.png",
                 {"attribution": "&copy; OpenTopoMap contributors"})
    })

    # Store drawn shapes (GeoJSON)
    drawn_shapes = param.List(default=[])

    _template = "<div id='map' style='${container_style}'></div>"

    _scripts = {
        'render': """
            // Init map
            state.map = L.map(map, data.map_options).setView([data.center[0], data.center[1]], data.zoom);

            // Base layers
            state.baseLayers = {};
            for (const [name, entry] of Object.entries(data.tile_layers)) {
                const [url, opts] = entry;
                state.baseLayers[name] = L.tileLayer(url, opts);
            }
            const firstLayer = Object.values(state.baseLayers)[0];
            firstLayer.addTo(state.map);
            L.control.layers(state.baseLayers).addTo(state.map);

            // Scatter + hover marker
            state.pointsLayer = L.layerGroup().addTo(state.map);
            state.hover = L.circleMarker([0,0], {radius:8, color:'red', opacity:0.9});

            // Drawing layer
            state.drawnItems = new L.FeatureGroup().addTo(state.map);

            // Drawing controls
            state.drawControl = new L.Control.Draw({
                edit: { featureGroup: state.drawnItems },
                draw: {
                    polygon: true,
                    circle: true,
                    rectangle: true,
                    polyline: true,
                    marker: true
                }
            });
            state.map.addControl(state.drawControl);

            // Measurement control
            state.measure = new L.Control.Measure({
                primaryLengthUnit: 'meters',
                secondaryLengthUnit: 'kilometers',
                primaryAreaUnit: 'sqmeters',
                secondaryAreaUnit: 'hectares'
            });
            state.measure.addTo(state.map);

            // Capture draw events
            state.map.on(L.Draw.Event.CREATED, function (e) {
                const layer = e.layer;
                state.drawnItems.addLayer(layer);
                const geo = layer.toGeoJSON();
                data.drawn_shapes = [...data.drawn_shapes, geo];
            });

            state.map.on('draw:deleted', function(e) {
                data.drawn_shapes = state.drawnItems.toGeoJSON().features;
            });

            // Hide hover marker on leave
            map.addEventListener('mouseleave', () => {
                if (state.map.hasLayer(state.hover)) state.map.removeLayer(state.hover);
            });
        """,

        'after_layout': "state.map.invalidateSize();",

        'lats': """
            state.pointsLayer.clearLayers();
            if (data.lats.length === 0) return;

            const vals = data.values;
            const vmin = Math.min(...vals);
            const vmax = Math.max(...vals);
            const palette = Bokeh.Palettes[data.cmap];

            for (let i=0; i<data.lats.length; i++) {
                const norm = (vals[i] - vmin) / (vmax - vmin + 1e-9);
                const ci = Math.floor(norm * (palette.length-1));
                const color = palette[ci];
                L.circleMarker([data.lats[i], data.lons[i]], {
                    radius:5, fillColor:color, color:color,
                    fillOpacity:0.8, opacity:0.8
                }).addTo(state.pointsLayer);
            }
        """,

        'marker': """
            if (data.marker && data.show_marker) {
                const lat = data.marker[0], lon = data.marker[1];
                state.hover.setLatLng([lat, lon]);
                if (!state.map.hasLayer(state.hover)) state.map.addLayer(state.hover);
            }
        """,

        'show_marker': """
            if (data.show_marker && data.marker) {
                const lat = data.marker[0], lon = data.marker[1];
                state.hover.setLatLng([lat, lon]);
                if (!state.map.hasLayer(state.hover)) state.map.addLayer(state.hover);
            } else {
                if (state.map.hasLayer(state.hover)) state.map.removeLayer(state.hover);
            }
        """
    }

    _extension_name = 'leaflet'

    __css__ = [
        'https://unpkg.com/leaflet@1.9.4/dist/leaflet.css',
        'https://unpkg.com/leaflet-draw/dist/leaflet.draw.css',
        'https://unpkg.com/leaflet-measure/dist/leaflet-measure.css'
    ]
    __javascript__ = [
        'https://unpkg.com/leaflet@1.9.4/dist/leaflet.js',
        'https://unpkg.com/leaflet-draw/dist/leaflet.draw.js',
        'https://unpkg.com/leaflet-measure/dist/leaflet-measure.js'
    ]


pn.extension('leaflet')

In [3]:
import numpy as np
import panel as pn

pn.extension('leaflet')

# Example data
n = 50
lat = 37.3 + 0.05*np.sin(np.linspace(0, 2*np.pi, n))
lon = -122.0 + 0.05*np.cos(np.linspace(0, 2*np.pi, n))
values = np.linspace(0, 1, n)

leaf = LeafletScatterFull(
    lats=list(map(float, lat)),
    lons=list(map(float, lon)),
    values=list(map(float, values)),
    cmap="Inferno256",
    container_style="width:600px;height:400px;border:1px solid black;"
)

pn.Row(leaf, pn.pane.JSON(leaf.param.drawn_shapes, depth=3)).servable()


BokehModel(combine_events=True, render_bundle={'docs_json': {'be6bf4e6-c3c6-4a8d-a866-c7f3142bb622': {'version…